# `RankingPipeline`

Dans ce script on teste la pipeline complète.

## Calcul des scores

La méthode `run` lance le calcul des scores.

In [1]:
import os
from getpass import getpass

os.environ["HF_DATASETS_CACHE"] = "~/.cache/huggingface/datasets/"
os.environ["HF_TOKEN"] = getpass("Enter your HuggingFace token:")

In [2]:
from rank_comparia.pipeline import RankingPipeline

In [ ]:
pipeline = RankingPipeline(
    method="elo_random",
    include_votes=True,
    include_reactions=True,
    bootstrap_samples=5,
    batch=False,
    export_graphs=False,
)

In [4]:
pipeline.matches

model_a,model_b,score,conversation_pair_id,categories
str,str,i32,str,list[str]
"""gemma-3-12b""","""claude-3-7-sonnet""",2,"""6eabbea23d974d4ea371536a485011…","[""Culture & Cultural geography"", ""Education""]"
"""gemma-3-4b""","""ministral-8b-instruct-2410""",1,"""94312ca8b095489aabb607db75046f…","[""Natural Science & Formal Science & Technology"", ""Education""]"
"""gpt-4o-2024-08-06""","""llama-3.3-70b""",0,"""ef11e65566f64078ba37dffd3f910b…","[""Natural Science & Formal Science & Technology"", ""Education""]"
"""gpt-4o-mini-2024-07-18""","""gemini-1.5-pro-002""",1,"""a4821ca5ff0e4caf8ed848f8dea655…","[""Business & Economics & Finance""]"
"""gpt-4o-mini-2024-07-18""","""qwq-32b""",0,"""56d0d140a843414a929d13a120df1a…","[""Health & Wellness & Medicine"", ""Food & Drink & Cooking"", ""Sports""]"
…,…,…,…,…
"""gemma-3-12b""","""claude-3-7-sonnet""",0,"""a8fad8b5a5d5451289d4c74c757d2b…","[""Natural Science & Formal Science & Technology"", ""Other""]"
"""chocolatine-2-14b-instruct-v2.…","""mistral-nemo-2407""",2,"""108af08538a147c98adbe457c36743…","[""Law & Justice"", ""Business & Economics & Finance"", ""Politics & Government""]"
"""llama-3.1-8b""","""llama-3.3-70b""",1,"""946a292204f5485a8404d18c83cba3…","[""Entertainment & Travel & Hobby"", ""Personal Development & Human Resources & Career"", ""Sports""]"


In [5]:
pipeline.match_list()

[Match(model_a='gemma-3-12b', model_b='claude-3-7-sonnet', score=<MatchScore.A: 2>, id='6eabbea23d974d4ea371536a4850114d-66b95b9fd188451dbb607c4566cf0604'),
 Match(model_a='gemma-3-4b', model_b='ministral-8b-instruct-2410', score=<MatchScore.Draw: 1>, id='94312ca8b095489aabb607db75046f57-ea012c34a6d04b7e9c70dc2e0d49dbf5'),
 Match(model_a='gpt-4o-2024-08-06', model_b='llama-3.3-70b', score=<MatchScore.B: 0>, id='ef11e65566f64078ba37dffd3f910b1b-98c2e479cd3541218d0ce5d9670b4104'),
 Match(model_a='gpt-4o-mini-2024-07-18', model_b='gemini-1.5-pro-002', score=<MatchScore.Draw: 1>, id='a4821ca5ff0e4caf8ed848f8dea655f6-a54c36a6df554dc39962d95df00d6a18'),
 Match(model_a='gpt-4o-mini-2024-07-18', model_b='qwq-32b', score=<MatchScore.B: 0>, id='56d0d140a843414a929d13a120df1af9-5c39c67f8cba4f5db4eaed398bb9b7c5'),
 Match(model_a='deepseek-r1', model_b='mistral-small-24b-instruct-2501', score=<MatchScore.B: 0>, id='dc99cdfacd29466fbf4bfeaf94fc0e3f-8a6215d491a14f5387f411d825838709'),
 Match(model_a=

In [6]:
scores = pipeline.run()

Computing bootstrap scores from a sample of 99668 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:01<00:00,  4.26it/s]


In [7]:
scores

model,median,p2.5,p97.5
str,f64,f64,f64
"""gemini-2.0-flash-exp""",1164.178335,1081.067884,1172.002278
"""gemini-2.0-flash-001""",1154.48751,1053.697866,1181.836921
"""deepseek-v3-chat""",1145.843132,1034.640564,1185.983647
"""deepseek-v3-0324""",1122.350396,1043.255113,1210.358801
"""gemma-3-27b""",1122.281755,1071.682263,1149.156084
…,…,…,…
"""mixtral-8x22b-instruct-v0.1""",838.373958,811.308,957.471374
"""chocolatine-2-14b-instruct-v2.…",829.718727,783.626276,861.346911
"""chocolatine-14b-instruct-dpo-v…",760.246558,725.779745,831.008609


Uniquement avec les données de votes.

In [ ]:
pipeline = RankingPipeline(
    method="elo_random",
    include_votes=True,
    include_reactions=False,
    bootstrap_samples=5,
    batch=False,
    export_graphs=False,
)
scores_votes = pipeline.run()

In [9]:
scores_votes

model,median,p2.5,p97.5
str,f64,f64,f64
"""gemini-2.0-flash-001""",1207.752512,1068.822558,1228.048278
"""gemma-3-27b""",1150.953358,1103.044527,1167.140821
"""gemini-2.0-flash-exp""",1114.932443,1015.992254,1208.771878
"""llama-3.1-nemotron-70b-instruc…",1114.771051,1029.391468,1150.474981
"""command-a""",1109.698839,1048.896991,1171.78612
…,…,…,…
"""mixtral-8x22b-instruct-v0.1""",836.2201,764.431421,941.775994
"""chocolatine-2-14b-instruct-v2.…",833.782019,815.249277,889.656596
"""Yi-1.5-9B-Chat""",785.956156,751.56039,919.597839


## Pipeline avec un ranker alternatif

In [ ]:
pipeline = RankingPipeline(
    method="ml", include_votes=True, include_reactions=True, bootstrap_samples=5, batch=False, export_graphs=False
)

In [13]:
scores_ml = pipeline.run()

Computing bootstrap scores from a sample of 99668 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:01<00:00,  4.14it/s]


In [ ]:
pipeline = RankingPipeline(
    method="ml", include_votes=True, include_reactions=False, bootstrap_samples=5, batch=False, export_graphs=False
)
scores_ml_votes = pipeline.run()

## Comparaison des différentes méthodes

In [18]:
import polars as pl

pl.concat(
    [
        scores.select("model", "median").rename(mapping={"median": "score_elo"}),
        scores_votes.select("model", "median").rename(mapping={"median": "score_elo_votes"}),
        scores_ml.select("model", "median").rename(mapping={"median": "score_ml"}),
        scores_ml_votes.select("model", "median").rename(mapping={"median": "score_ml_votes"}),
    ],
    how="align",
)

model,score_elo,score_elo_votes,score_ml,score_ml_votes
str,f64,f64,f64,f64
"""Yi-1.5-9B-Chat""",750.037402,785.956156,764.439315,781.177538
"""aya-expanse-32b""",1020.667367,992.234386,1010.366967,1014.978885
"""aya-expanse-8b""",1040.180811,977.224125,995.75911,1002.849172
"""c4ai-command-r-08-2024""",959.927073,970.259973,955.26738,959.454433
"""chocolatine-14b-instruct-dpo-v…",760.246558,770.125597,792.652056,780.745574
…,…,…,…,…
"""qwen2.5-32b-instruct""",985.362676,931.052784,989.71134,984.661056
"""qwen2.5-7b-instruct""",930.997033,948.537077,945.307286,964.380778
"""qwen2.5-coder-32b-instruct""",1009.796101,896.436785,955.788005,960.282409


In [ ]:
import polars as pl
import altair as alt

df_pl = pl.concat(
    [
        scores.select("model", "median").rename(mapping={"median": "score_elo"}),
        scores_votes.select("model", "median").rename(mapping={"median": "score_elo_votes"}),
        scores_ml.select("model", "median").rename(mapping={"median": "score_ml"}),
        scores_ml_votes.select("model", "median").rename(mapping={"median": "score_ml_votes"}),
    ],
    how="align",
).sort("score_elo", descending=True)

df = df_pl.to_pandas()
df_long = df.melt(
    id_vars=["model"],
    value_vars=["score_elo", "score_elo_votes", "score_ml", "score_ml_votes"],
    var_name="score_type",
    value_name="score",
)
legend_labels = {
    "score_elo": "Elo score (all data)",
    "score_elo_votes": "Elo score (votes data)",
    "score_ml": "BT score (all data)",
    "score_ml_votes": "BT score (votes data)",
}
df_long["score_type"] = df_long["score_type"].map(legend_labels)

chart = (
    alt.Chart(df_long)
    .mark_circle(size=80)
    .encode(
        x=alt.X("model:N", sort=df["model"].tolist(), title="Model"),
        y=alt.Y("score:Q", title="Score", scale=alt.Scale(domain=[500, 1300])),
        color=alt.Color("score_type:N", title="Score Type"),
        tooltip=["model", "score", "score_type"],
    )
    .properties(width=600, height=400)
)

chart

alt.Chart(...)

In [27]:
chart.save("scores_comparison.png", ppi=300)

## Scores par catégorie

Les méthodes `run_category` et `run_all_categories` permettent de calculer des scores pour une catégorie spécifiée ou pour toutes les catégories (avec un nombre de matchs total supérieur à un seuil).

In [ ]:
pipeline = RankingPipeline(
    method="elo_random",
    include_votes=True,
    include_reactions=True,
    bootstrap_samples=5,
    batch=False,
    export_graphs=False,
)

In [ ]:
pipeline.run_category("Education")

In [ ]:
results = pipeline.run_all_categories()